# scrap repo stats using PyGitHub

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import os
os.environ['db_user']='zz3hs'
os.environ['db_pwd']='zz3hs'

In [6]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from github import Github, RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
# using an access token
access_token = "e0386eeea5051b22f309154e8dee27fb3657853e"

In [8]:
# connect to the database, download data, limit to top 5000 repos that have the highest number of commits
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked WHERE (commits BETWEEN '10000' AND '11000')'''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
print(raw_slug_data.head())
print(raw_slug_data.shape)
print(raw_slug_data.isna().sum())

raw_slugs = raw_slug_data["slug"].tolist()
#slugs = [raw_slugs.strip(' ') for raw_slugs in slugs]
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())
    
print(len(slugs))
#print(slugs)

                                 id          spdx                    slug  \
0  MDEwOlJlcG9zaXRvcnkxOTE5MDY2Mjg=           MIT    TheKiteRunners/react   
1  MDEwOlJlcG9zaXRvcnkxOTkyNzg2NzQ=  BSD-3-Clause  pooog90001/fence-gates   
2  MDEwOlJlcG9zaXRvcnkxMzAwODMyNTQ=           MIT    WhitePrime/storybook   
3  MDEwOlJlcG9zaXRvcnk1NjAwMzA2Mw==       GPL-2.0     chrismbarnes/ndnSIM   
4  MDEwOlJlcG9zaXRvcnkyNTEzNjYyMg==    Apache-2.0        AtwoodCSB/Docker   

            createdat                                        description  \
0 2019-06-14 08:33:56                                       源码解析基于16.8.6   
1 2019-07-28 11:27:22                                               None   
2 2018-04-18 15:26:24                                               None   
3 2016-04-11 19:54:59  Extension of ns3 module to implement different...   
4 2014-10-13 00:36:40                                             Docker   

  primarylanguage                                            branch  commits  \


In [ ]:
#practice slugs
slugs = ["moderndive/ModernDive_book", "crystalzang/mpred", "rrrrrrrrr"]

In [9]:
def pull_repo_stats():
    df_repo_stats = pd.DataFrame()
    for slug in slugs:
        while True:
            try:
                #if false, retry until true, max number of retry is 20 times
                g = Github(access_token, retry = 20, timeout = 15)
                repo = g.get_repo(slug)
                df_repo_stats = df_repo_stats.append({
                    "slug": slug,
                    'stars': repo.stargazers_count,
                    'watchers': repo.subscribers_count,
                    'topics': repo.get_topics()
                }, ignore_index = True)
            except RateLimitExceededException as e:
                print(e.status)
                print('Rate limit exceeded --', slug)
                time.sleep(300)
                continue
            except BadCredentialsException as e:
                print(e.status)
                print('Bad credentials exception --', slug)
                break
            except UnknownObjectException as e:
                print(e.status)
                print('Unknown object exception --', slug)
                break
            except GithubException as e:
                print(e.status)
                print('General exception --', slug)
                break
            except requests.exceptions.ConnectionError as e:
                print('Retries limit exceeded --', slug)
                print(str(e))
                time.sleep(10)
                continue
            except request.exceptions.Timeout as e:
                print('Time out exception --', slug)
                print(str(e))
                time.sleep(10)
                continue
            break
        df_repo_stats.to_csv(r'/home/zz3hs/git/dspg21oss/data/repo_stats.csv', index = False)        

In [10]:
pull_repo_stats()

404
Unknown object exception -- RawHawk11/11kwaHwaR
404
Unknown object exception -- michaeltulagan/material-ui-clone
404
Unknown object exception -- LorenzReinhart/ONOSnew
404
Unknown object exception -- chrisi21/sickchill
404
Unknown object exception -- bootstraponline-archive/gerrit-mirror
404
Unknown object exception -- yarlevi/openshift
404
Unknown object exception -- cyre1450/jmglz
404
Unknown object exception -- jondavidy/psi4-clone
404
Unknown object exception -- zanone/kolla-ansible
404
Unknown object exception -- simplechain-org/simplechain
404
Unknown object exception -- OpenVisionE2/NCam
404
Unknown object exception -- lucerattow/botpress
404
Unknown object exception -- krishnamani/tryghost
404
Unknown object exception -- cybel/uclibc-ng
404
Unknown object exception -- Haslik/ghost
404
Unknown object exception -- ymmah/turbogeth
404
Unknown object exception -- emergenetwork/go-ethereum
404
Unknown object exception -- daniil-besedin/jekyll
404
Unknown object exception -- mika

In [85]:
df_repo_stats = pd.read_csv(r'/home/zz3hs/git/dspg21oss/output/repo_stats.csv')

In [86]:
print(df_repo_stats.head())
print(df_repo_stats)

                               slug  stars topics  watchers
0                Rafael3086/spryker    NaN    NaN       NaN
1                    jplyley/PDF.js    NaN    NaN       NaN
2           jcoliver/OSUScratchpads    NaN    NaN       NaN
3                  simonp22/AnkhSVN    NaN    NaN       NaN
4  AlenTuzla/anycubici3megas3dtouch    NaN    NaN       NaN
                                  slug  stars topics  watchers
0                   Rafael3086/spryker    NaN    NaN       NaN
1                       jplyley/PDF.js    NaN    NaN       NaN
2              jcoliver/OSUScratchpads    NaN    NaN       NaN
3                     simonp22/AnkhSVN    NaN    NaN       NaN
4     AlenTuzla/anycubici3megas3dtouch    NaN    NaN       NaN
...                                ...    ...    ...       ...
5459                               NaN    0.0     []       1.0
5460                               NaN    0.0     []       0.0
5461                               NaN    0.0     []       1.0
5462      

In [88]:
df_repo_stats.isna().sum()

slug        2715
stars       2749
topics      2749
watchers    2749
dtype: int64

In [89]:
df_repo_stats.shape

(5464, 4)